### Name :- Pranit Duta
### QUESTION 2 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from scipy.stats import multivariate_normal as mvn

In [6]:
data=pd.read_csv("/Users/pranitdutta/Downloads/Submission/points.dat.txt", header=None, dtype=np.float64, delimiter=' ')

In [7]:
data.shape

(1000, 2)

In [9]:
data=np.array(data)

In [10]:
print(type(data))

<class 'numpy.ndarray'>


In [11]:
train, test = data[:int(0.7*len(data))], data[int(0.7*len(data)):]

In [13]:
k = 2
m1 = np.random.multivariate_normal(np.mean(data, axis=0), np.cov(data.T), size=1).flatten()
m2 = np.random.multivariate_normal(np.mean(data, axis=0), np.cov(data.T), size=1).flatten()
cov1 = np.cov(data.T)
cov2 = np.cov(data.T)
initpi = 0.5
nIter = 50

In [14]:
mu = np.array([m1, m2], np.float64)

In [16]:
sigma = np.array([cov1, cov2], np.float64)

In [17]:
print(sigma)

[[[ 7.9328436  -0.10559592]
  [-0.10559592  6.2470177 ]]

 [[ 7.9328436  -0.10559592]
  [-0.10559592  6.2470177 ]]]


In [22]:
def Expectation(data, k, pi, mu, sigma):
    mu1=mu[0]
    mu2=mu[1]
    cov1=sigma[0]
    cov2=sigma[1]
    mu1.reshape(2,)
    mu2.reshape(2,)
    
    gauss2 = mvn.pdf(data, mean=mu2, cov=cov2)
    gauss1 = mvn.pdf(data, mean=mu1, cov=cov1)
    w1 = pi * gauss2
    w2 = (1-pi) * gauss1
    gamma = w1/(w1+w2)
    r = np.concatenate([w2.reshape(len(data), 1), w1.reshape(len(data), 1)], axis=1)
    r = r / np.sum(r, axis=1, keepdims=True)
    return(data, k, gamma,r)

In [29]:
def Maximization(data, k, gamma):
    	# Maximization of Means
    NumMu1, DenMu1, NumMu2, DenMu2 = 0, 0, 0, 0
    for i in range(0,len(data)):
        NumMu1 += (1-gamma[i]) * data[i]
        DenMu1 += (1-gamma[i])

        NumMu2 += gamma[i] * data[i]
        DenMu2 += gamma[i]

    mu1 = NumMu1/DenMu1
    mu2 = NumMu2/DenMu2

    # Maximization of Covariances
    NumCov1, DenCov1, NumCov2, DenCov2 = 0, 0, 0, 0
    
    for i in range(0,len(data)):
        val1 = np.matrix(data[i]-mu1)
        NumCov1 += (1-gamma[i]) * np.dot(val1.T, val1)
        DenCov1 += (1-gamma[i])

        val2 = np.matrix(data[i]-mu2)
        NumCov2 += gamma[i] * np.dot(val2.T, val2)
        DenCov2 += gamma[i]

    sigma1 = NumCov1/DenCov1
    sigma2 = NumCov2/DenCov2

    # Maximization of Mixture weights pi
    pi = sum(gamma)/len(data)
    
    return mu1, mu2, sigma1, sigma2, pi

In [30]:
def EM(data, k, initpi, mu, sigma, nIter):
    for i in range(0,nIter):
        x,k,gamma,r = Expectation(data, k, initpi, mu, sigma)
        mu1, mu2, sigma1, sigma2, pi = Maximization(data, k, gamma)
        mu = np.array([mu1, mu2], np.float64)
        sigma = np.array([sigma1, sigma1], np.float64)
        initialpi = pi
    print('mu')
    print(mu)
    print('sigma')
    print(sigma)
    print('pi')
    print(initialpi)

In [31]:
EM(data, k, initpi, mu, sigma, nIter)

mu
[[-1.71090222  0.79402818]
 [ 0.90997453 -3.25652284]]
sigma
[[[6.33868697 2.79545334]
  [2.79545334 2.49199601]]

 [[6.33868697 2.79545334]
  [2.79545334 2.49199601]]]
pi
0.33976171243793857
